# Chapter 18. asyncio를 이용한 동시성

* 동시성은 구조에 관한 것으로서 한 번에 많은 것을 다룬다. 
* 병렬성은 실행에 관한 것으로서 한 번에 많은 것을 한다.

이 장에서는 이벤트 루프에 의해 운용되는 코루틴을 이용해서 동시성을 구현하는 `asyncio` 패키지에 대해 설명한다. 

* 간단한 스레드 프로그램과 그에 준하는 `asyncio` 버전을 비교하면서 스레드와 비동기 작업의 관계를 보여준다.
* `asyncio.Future` 클래스와 `concurrent.futures.Future` 클래스의 차이점을 설명한다.
* 17장에서 구현한 국기 내려받기 예제의 비동기 버전을 구현한다.
* 스레드나 프로세스를 사용하지 않고 비동기 프로그래밍이 네트워크 프로그램에서 높은 동시성을 관리하는 방법을 설명한다.
* 코루틴으로 비동기 프로그래밍을 하기 위한 콜백을 개선시키는 방법을 설명한다.
* 블로킹 연산을 스레드 풀에 덜어줌으로써 이벤트 루프를 블로킹하지 않는 방법을 알아본다.
* `asyncio` 서버를 작성하고, 웹 애플리케이션의 높은 동시성을 다시 생각해본다.
* `asyncio`가 파이썬 생태계에서 커다란 영향력을 줄 수밖에 없는 이유를 설명한다.

## 18.1 스레드와 코루틴의 비교

```python
# spinner_thread.py

import threading
import itertools
import time
import sys

class Signal: # 외부에서 스레드를 제어하기 위해 사용할 go 속성만 있는 가변 객체
    go = True

def spin(msg, signal):
    write, flush = sys.stdout.write, sys.stdout.flush
    for char in itertools.cycle('|/-\\'): # 무한 루프
        status = char + ' ' + msg
        write(status)
        flush()
        write('\x08' * len(status)) # 문자열 길이만큼 백스페이스 문자를
        # 반복해서 앞으로 이동 시킴
        time.sleep(.1)
        if not signal.go: # go == False 면 루프 종료
            break
    # 공백 문자로 덮어쓰고 다시 커서를 처음으로 이동해서 메세지 출력행 청소
    write(' ' * len(status) + '\x08' * len(status))

def slow_function():
    # 입출력을 위해 장시간 기다리는 것처럼 보이게 만든다.
    time.sleep(3)
    # 주 스레드에서 sleep() 함수를 호출할 때 GIL이 해제되므로
    # 두번째 스레드가 진행
    return 42

def supervisor(): 
    # 이 함수는 두번째 스레드를 만들고, 스레드 객체를 출력하고,
    # 시간이 오래 걸리는 연산을 수행하고 나서 스스로 스레드 제거
    signal = Signal()
    spinner = threading.Thread(target=spin,
                               args=('thinking!', signal))
    print('spinner object:', spinner) # 두번째 스레드 객체 출력
    spinner.start() # 두번째 스레드 실행
    result = slow_function() # slow_function() 실행
    # 그러면 주 스레드가 블로킹되고 그동안 두번째 스레드가 
    # 텍스트 스피너 애니메이션을 보여준다.
    signal.go = False # signal의 상태 변경
    spinner.join() # spinner 스레드가 끝날 때까지 기다린다
    return result

def main():
    result = supervisor() # supervisor() 실행
    print('Answer:', result)

if __name__ == '__main__':
    main()
```

```python
# spinner_asyncio.py

import asyncio
import itertools
import sys

@asyncio.coroutine # asyncio에 사용할 코루틴은 @asyncio.coroutine으로 데커레이트해야한다.
def spin(msg): # signal 인수 불필요
    write, flush = sys.stdout.write, sys.stdout.flush
    for char in itertools.cycle('|/-\\'):
        status = char + ' ' + msg
        write(status)
        flush()
        write('\x08' * len(status)) 
        try:
            yield from asyncio.sleep(.1) # 이벤트 루프를 블로킹하지 않고 잠자기 위해
            # time.sleep 대신 yield from asyncio.sleep 사용
        except asyncio.CancelledError: # spin()이 깨어난 후 
            # asyncio.CancelledError 예외가 발생하면 루프 종료
            break
    write(' ' * len(status) + '\x08' * len(status))

@asyncio.coroutine 
def slow_function():
    # slow_function()은 이제 코루틴으로서, 코루틴이 잠자면서 입출력을 수행하는 체 하는 동안
    # 이벤트 루프가 진행될 수 있게 하기 위해 yield from을 사용한다.
    yield from asyncio.sleep(3) # 메인 루프의 제어 흐름을 처리한다
    return 42

@asyncio.coroutine 
def supervisor(): 
    # supervisor()도 코루틴이므로 yield from을 이용해 slow_function() 구동 가능
    # asyncio.async()는 spin() 코루틴의 실행을 스케줄링하고 Task 객체 안에 넣어 즉시 반환
    spinner = asyncio.async(spin('thinking!')) 
    print('spinner object:', spinner) # Task 객체 출력
    result = yield from slow_function() # 함수를 구동해서 완료되면 반환된 값 가져온다
    # 그동안에 이벤트 루프는 계속 실행된다. slow_function()이 yield from asyncio.sleep()을 실행해서
    # 메인 루프로 제어권을 넘기기 때문이다.
    spinner.cancel() # Task객체는 cancel() 메서드를 호출해서 취소할 수 있다.
    # 그러면 코루틴이 중단된 곳의 yield from에서 asyncio.CancelledError 예외가 발생한다.
    # 코루틴은 예외를 잡아서 지연시키거나 취소 요청을 거부할 수 있다.
    return result

def main():
    loop = asyncio.get_event_loop() # 이벤트 루프에 대한 참조를 가져온다.
    result = loop.run_until_complete(supervisor()) # supervisor() 코루틴을 구동해서 완료한다.
    loop.close()
    print('Answer:', result)

if __name__ == '__main__':
    main()
```

```python
# 3.5 버전에 추가된 async와 await 키워드 이용한 구현

import asyncio
import itertools


async def spin(msg):  # <1>
    for char in itertools.cycle('|/-\\'):
        status = char + ' ' + msg
        print(status, flush=True, end='\r')
        try:
            await asyncio.sleep(.1)  # <2>
        except asyncio.CancelledError:  # <3>
            break
    print(' ' * len(status), end='\r')


async def slow_function():  # <4>
    # pretend waiting a long time for I/O
    await asyncio.sleep(3)  # <5>
    return 42


async def supervisor():  # <6>
    spinner = asyncio.create_task(spin('thinking!'))  # <7>
    print('spinner object:', spinner)  # <8>
    result = await slow_function()  # <9>
    spinner.cancel()  # <10>
    return result


def main():
    result = asyncio.run(supervisor())  # <11>
    print('Answer:', result)


if __name__ == '__main__':
    main()
```

반드시 `@asyncio.coroutine` 데커레이터를 사용해야 하는 것은 아니지만, 되도록이면 사용을 권고한다. 이는 이 데커레이터가 코루틴을 일반 함수와 다르게 보이도록 만들며, 코루틴이 yield from 되지 않고(즉 일부작업이 완료되지 않았으므로 버그 발생 가능성이 높음) 가비지 컬렉트되는 경우 경고 메시지를 출력하므로 디버깅에 도움이 된다.

두 예제의 `supervisor()` 함수의 주요 차이점은 다음과 같다.

* `asyncio.Task`는 `threading.Thread`와 거의 대등하다.
* `Task`는 코루틴을 구동하고, `Thread`는 콜러블을 호출한다.
* `Task` 객체는 직접 생성하지 않고, 코루틴을 `asyncio.async()`나 `loop.create_task()`에 전달해서 가져온다.
* `Task` 객체를 가져오면 이 객체는 이미 `asyncio.async()` 등에 의해 실행이 스케줄링되어 있다. `Thread` 객체는 `start()` 메서드를 호출해서 실행하라고 명령해야 한다.
* 스레드화된 `supervisor()`에서 `slow_function()`은 평범한 함수로서, 스레드에 의해 직접 호출된다. 비동기 `supervisor()`에서 `slow_function()`은 yield from으로 구동하는 코루틴이다.
* 스레드는 외부에서 API를 이용해서 중단시킬 수 없다. `Task`는 `cancel()` 메서드를 통해 CancelledError를 발생시키면 코루틴은 중단되었던 yield 문에서 예외를 잡아서 처리할 수 있다.
* `supervisor()` 코루틴은 `main()` 함수 안에서 `loop.run_until_complete()`로 실행해야 한다.

코루틴은 모든 것이 인터럽트로부터 보호된다. 여러 스레드의 연산을 동기화하기 위해 락을 잠그는 대신, 언제든 실행되고 있는 코루틴 중 하나만 사용하면 된다. 

### 18.1.1 asyncio.Future: 논블로킹 설계

`asyncio.Future`와 `concurrent.futures.Future` 클래스는 인터페이스가 거의 같지만 다르게 구현되어 있으므로 바꿔 쓸 수 없다. `asyncio.Future` 클래스도 `done()`, `add_done_callback()`, `result()` 등의 메서드를 제공한다. 앞 두개는 17.1.3절에서 설명한 대로 동작하지만 `result()`는 매우 다르며, 굳이 쓰지 말고 `yield from`을 쓰자...

* 코루틴 안에서 my_future가 실행을 완료한 다음에 수행할 작업은 단순히 yield from my_future 뒤에 넣으면 되므로 my_future.add_done_callback()을 호출할 필요가 없다.
* my_future에 대한 yield from 표현식의 값이 result가 되므로 my_future.result()를 호출할 필요가 없다.

### 18.1.2 Future, Task, 코루틴에서 생성하기

`foo()`가 코루틴 함수거나(즉, 호출되면 코루틴 객체 반환) Future나 Task 객체를 반환하는 일반 함수면 `res = yield from foo()` 코드가 작동한다. 코루틴을 받아서 Task 객체를 가져오기 위해서는 주로 다음 두 방법을 사용한다.

* `asyncio.async(coro_or_future, *, loop=None)` : 첫번째 인수가 Future나 Task 객체면 그대로 반환되며, 코루틴이면 `async()`가 `loop.create_task()`를 호출해서 Task를 생성한다. loop 키워드 인수에 이벤트 루프를 생략하면 `async()`가 `asyncio.get_event_loop()`를 호출해서 루프 객체를 가져온다.
* `BaseEventLoop.create_task(coro)` : 생략...

내부적으로 `asyncio.async()`를 이용해서 받은 코루틴을 자동으로 `asyncio.Task` 객체 안에 래핑하는 함수들이 많이 있으며, 대표적으로 `BaseEventLoop.run_until_complete()`다. 파이썬 콘솔에서 Future 객체나 코루틴을 실험하고 싶다면 다음 코드를 사용한다.

```python
import asyncio
def run_sync(coro_or_future):
    loop = asyncio.get_event_loop()
    return loop.run_until_complete(coro_or_future)

a = run_sync(some_coroutine())
```

## 18.2 asyncio와 aiohttp로 내려받기

asyncio는 TCP와 UDP만 직접 지원한다. HTTP 등의 프로토콜을 지원하려면 aiohttp 등의 서드파티 패키지를 사용해야 한다. 

국기 이미지를 내려받는 flag_asyncio.py 스크립트의 전체 흐름은 다음과 같다.

1. `download_one()`을 호출해서 생성된 여러 코루틴 객체를 이벤트 루프에 넣어 `download_many()` 안에서 프로세스를 시작한다.
1. `asyncio` 이벤트 루프는 각각의 코루틴을 차례대로 활성화한다.
1. `get_flag()` 등의 클라이언트 코루틴이 `aiohttp.request()` 등의 라이브러리 코루틴에 위임하기 위해 yield from을 사용하면, 제어권이 이벤트 루프로 넘어가서 이벤트 루프가 이전에 스케줄링된 다른 코루틴을 실행할 수 있게 된다.
1. 블로킹된 연산이 완료되었을 때 통지받기 위해 이벤트 루프는 콜백에 기반한 저수준 API를 사용한다.
1. 연산이 완료되면 메인 루프는 결과를 중단된 코루틴에 보낸다.
1. 그러고 나면 코루틴이 예를 들면 `get_flag()`의 `yield from resp.read()`와 같은 다음 yield from 문으로 넘어간다. 이제 이벤트 루프가 다시 제어권을 가져오고, 종료될 때까지 4,5,6단계를 반복한다.

```python
import asyncio

import aiohttp  

from flags import BASE_URL, save_flag, show, main  # flags 모듈에서 구현한 일부 함수 재사용


@asyncio.coroutine  # 코루틴 데커레이트
def get_flag(cc):
    url = '{}/{cc}/{cc}.gif'.format(BASE_URL, cc=cc.lower())
    resp = yield from aiohttp.request('GET', url)  # 블로킹 연산은 코루틴으로 구현되었고 
    # yield from을 이용해서 이 코루틴에 위임하면 코루틴이 비동기식으로 실행된다.
    image = yield from resp.read()  # 응답 내용을 읽는 것은 별동의 비동기 연산에서 구현한다.
    return image


@asyncio.coroutine
def download_one(cc):  
    image = yield from get_flag(cc)  # 단어들을 yield from으로 가져온다는 것이 차이점
    show(cc)
    save_flag(image, cc.lower() + '.gif')
    return cc


def download_many(cc_list):
    loop = asyncio.get_event_loop()  # 하위 이벤트 루프 구현에 대한 참조를 가져온다.
    to_do = [download_one(cc) for cc in sorted(cc_list)]  # 국기 이미지를 가져올 때마다
    # download_one() 함수를 호출해서 제너레이터 객체 리스트를 생성한다.
    wait_coro = asyncio.wait(to_do)  # wait()는 블로킹 함수가 아니며, 일종의 코루틴으로
    # 자신에게 전달된 코루틴들이 모두 완료되면 완료된다.
    res, _ = loop.run_until_complete(wait_coro)  # wait_coro()가 완료될 떄까지 이벤트 루프 실행
    # 이 부분은 이벤트 루프가 실행하는 동안 블로킹된다
    loop.close() # 이벤트 루프 종료

    return len(res)


if __name__ == '__main__':
    main(download_many)
```

`asyncio.wait()` 코루틴은 Future 객체나 코루틴의 반복형을 받고, `wait()`는 각 코루틴을 Task 안에 래핑한다. `wait()`는 코루틴 함수이기 때문에 이를 호출하면 코루틴/제너레이터 객체가 반환되며, 이 코루틴을 구동하기 위해 `loop.run_until_complete()`에 전달한다. 실행이 완료되면 (<실행 완료된 Future들의 집합>, <실행이 완료되지 않은 Future들의 집합>) 튜플을 반환한다(위 예제에서는 두번째 집합은 공집합).

`asyncio`에 대해 알아야 할 새로운 개념이 많지만, `yield from` 키워드가 없는 것처럼 생각하면 이해하기 쉽다. 예를 들면 

```python
@asyncio.coroutine
def get_flag(cc):
    url = '{}/{cc}/{cc}.gif'.format(BASE_URL, cc=cc.lower())
    resp = yield from aiohttp.request('GET', url)
    image = yield from resp.read()
    return image
```

코드는 블로킹되지 않는다는 점만 제외하면 다음 코드와 똑같이 작동한다.

```python
def get_flag(cc):
    url = '{}/{cc}/{cc}.gif'.format(BASE_URL, cc=cc.lower())
    resp = aiohttp.request('GET', url)
    image = resp.read()
    return image
```

`yield from foo` 구문을 사용하면 현재의 코루틴(즉 yield from 코드가 있는 대표 제너레이터)이 중단되지만, 제어권이 이벤트 루프로 넘어가고 이벤트 루프가 다른 코루틴을 구동할 수 있게 되므로 블로킹되지 않는다. foo가 Future 객체이든 코루틴이든 이 객체가 완료되면 결과를 중단된 코루틴으로 반환해서 실행을 계속하게 만든다.

`asyncio`와 `yield from`을 사용할 때 다음과 같은 특징이 있다.

* 코루틴 체인은 가장 바깥쪽 대표 제너레이터를 `loop.run_until_complete()` 등의 asyncio API에 전달함으로써 구동된다. 
* 가장 안쪽 하위 제너레이터는 실제로 입출력을 수행하는 라이브러리 함수(ex: resp = yield from aiohttp.request('GET', url))여야 한다.

## 18.3 블로킹 호출을 에둘러 실행하기